In [ ]:
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('./fta_pop_gdp_cleaned_v2.csv')
df_trades=pd.read_csv('./trades_v3.csv')
df_products=pd.read_csv('./products.csv')

In [ ]:

def get_query(row):
        str1 = 'MERGE (n1: Country {countryID : $n1}) ON CREATE SET n1.name = $n2,  '
        for num in df.columns[5:-3]:
            if row[num]==row[num]:
                str1+= f'n1.{num}= $n{num}, '
        str1 =str1[:-2]
        return str1

In [ ]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
          self.driver.close()

#####################################################################################################

    def create_country_node(self, row, has_fta):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(self._create_country_node, row, has_fta)

    @staticmethod
    def _create_country_node(tx, row, has_fta):
        query=(get_query(row))
        tx.run((query), n1=row['country_code'], n2=row['country'], npopulation=row['population'],
                n4=row['signed_year'], n5=row['inforced_year'],
                ngdp_1960=row['gdp_1960'],  ngdp_1961=row['gdp_1961'],  ngdp_1962=row['gdp_1962'],  ngdp_1963=row['gdp_1963'],  ngdp_1964=row['gdp_1964'],  ngdp_1965=row['gdp_1965'],
                ngdp_1966=row['gdp_1966'],  ngdp_1967=row['gdp_1967'],  ngdp_1968=row['gdp_1968'],  ngdp_1969=row['gdp_1969'],  ngdp_1970=row['gdp_1970'],  ngdp_1971=row['gdp_1971'],
                ngdp_1972=row['gdp_1972'],  ngdp_1973=row['gdp_1973'],  ngdp_1974=row['gdp_1974'],  ngdp_1975=row['gdp_1975'],  ngdp_1976=row['gdp_1976'],  ngdp_1977=row['gdp_1977'], 
                ngdp_1978=row['gdp_1978'],  ngdp_1979=row['gdp_1979'],  ngdp_1980=row['gdp_1980'],  ngdp_1981=row['gdp_1981'],  ngdp_1982=row['gdp_1982'],  ngdp_1983=row['gdp_1983'], 
                ngdp_1984=row['gdp_1984'],  ngdp_1985=row['gdp_1985'],  ngdp_1986=row['gdp_1986'],  ngdp_1987=row['gdp_1987'],  ngdp_1988=row['gdp_1988'],  ngdp_1989=row['gdp_1989'],
                ngdp_1990=row['gdp_1990'],  ngdp_1991=row['gdp_1991'],  ngdp_1992=row['gdp_1992'],  ngdp_1993=row['gdp_1993'],  ngdp_1994=row['gdp_1994'],  ngdp_1995=row['gdp_1995'], 
                ngdp_1996=row['gdp_1996'],  ngdp_1997=row['gdp_1997'],  ngdp_1998=row['gdp_1998'],  ngdp_1999=row['gdp_1999'],  ngdp_2000=row['gdp_2000'],  ngdp_2001=row['gdp_2001'],
                ngdp_2002=row['gdp_2002'],  ngdp_2003=row['gdp_2003'],  ngdp_2004=row['gdp_2004'],  ngdp_2005=row['gdp_2005'],  ngdp_2006=row['gdp_2006'],  ngdp_2007=row['gdp_2007'],
                ngdp_2008=row['gdp_2008'],  ngdp_2009=row['gdp_2009'],  ngdp_2010=row['gdp_2010'],  ngdp_2011=row['gdp_2011'],  ngdp_2012=row['gdp_2012'],  ngdp_2013=row['gdp_2013'],
                ngdp_2014=row['gdp_2014'],  ngdp_2015=row['gdp_2015'],  ngdp_2016=row['gdp_2016'],  ngdp_2017=row['gdp_2017'],  ngdp_2018=row['gdp_2018'],  ngdp_2019=row['gdp_2019'],
                ngdp_2020=row['gdp_2020'],  ngdp_2021=row['gdp_2021']  )

#####################################################################################################

    def FTA(self, p1, p2, p3, p4, p5, p6):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_fta, p1, p2, p3, p4, p5, p6)

    @staticmethod
    def load_fta(tx, p1, p2, p3, p4, p5, p6):

        query1 = (
            "MERGE (p1:FTA { ftaID: $p1 }) ON CREATE SET p1.has_fta = $p2, p1.signed_date=$p3, p1.date_inforce=$p4 "
        )

        query2=(
            "MATCH (p1:FTA { ftaID: $p1 }), (p5:Country { countryID: $p5 }) "
            "MERGE (p1)-[:hasFTA]->(p5) "
            )
        query3 = (
            "MATCH (p6:Country { countryID: $p6 }), (p1:FTA { ftaID: $p1 }) MERGE (p1)-[:hasFTA]->(p6) "
        )

        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)

##################################################################################################### 

    def product(self, p1, p2, p3, p4, p5, p6):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_product, p1, p2, p3, p4, p5, p6)

    @staticmethod
    def load_product(tx, p1, p2, p3, p4, p5, p6):

        query1=(
            "MERGE (p5:Product { productID: $p5 }) ON CREATE SET p5.product=$p6 "
            "MERGE (p3:Category { categoryID: $p3}) ON CREATE SET p3.Category = $p4 "
        )

        query2=(
            "MERGE (p1:Section { sectionID: $p1}) ON CREATE SET p1.section = $p2 "
        )

        query3=(
            "MATCH (p5:Product { productID: $p5 }), (p3:Category { categoryID: $p3 }) "
            "MERGE (p5)-[:hasCategory]->(p3) "
            )

        query4=(
            "MATCH (p1:Section { sectionID: $p1}), (p3:Category { categoryID: $p3 }) "
            "MERGE (p3)-[:hasSection]->(p1) "
            )

        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query4, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        
##################################################################################################### 

    def trade(self, p1, p2, p3, p4, p5, p6, p7, p8):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_trade, p1, p2, p3, p4, p5, p6, p7, p8)

    @staticmethod
    def load_trade(tx, p1, p2, p3, p4, p5, p6, p7, p8):

        query1=(
            "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "
            "MERGE (p4:Year { year: $p4 })"
        )

        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8)

#####################################################################################################


    def trade_rel(self, p1, p2, p3, p4, p5, p6, p7, p8):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_trade_rel, p1, p2, p3, p4, p5, p6, p7, p8)

    @staticmethod
    def load_trade_rel(tx, p1, p2, p3, p4, p5, p6, p7, p8):

        query3=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p2:Country { countryID: $p2 }) "
            "MERGE (p1)-[:exportedFrom]->(p2) "
            )

        query4=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p3:Country { countryID: $p3 }) "
            "MERGE (p1)-[:exportedTo]->(p3) "
            )

        query5=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p4:Year { year: $p4 }) "
            "MERGE (p1)-[:tradedYear]->(p4) "
            )

        query6=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p8:Section { sectionID: $p8 }) "
            "MERGE (p1)-[:tradedProduct]->(p8) "
            )


        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8)
        tx.run(query4, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8)
        tx.run(query5, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8)
        tx.run(query6, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8)

In [ ]:
df['signed_date'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['signed_year'] = pd.DatetimeIndex(df['signed_date']).year
df['date_inforce'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['inforced_year'] = pd.DatetimeIndex(df['date_inforce']).year
df['country_code']=df['country_code'].str.lower()
df['FTA_id']="fta_"+df['FTA_id'].astype(str)
# df.head(2)

In [ ]:
# df_trades['trade_id'] = df_trades['trade_id'].str.replace('ported','')
# df_trades['trade_id'] = df_trades['trade_id'].str.lower()
# # df_trades.head(5)

In [ ]:
if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://43b50553.databases.neo4j.io"
    user = "neo4j"
    password = "zRFUl83bppdZqwmO1DFBaAxVGvZOct9LETZ_HXzJZNg"
    app = App(uri, user, password)
    
    # df_trades=df_trades.head(25)

    for idx, row in df.iterrows():
      # if row['country_code']=="usa" or row['country_code']=="abw":  
        # app.create_country_node(row, False)
      app.create_country_node(row, False)
    
    for idx, row in df.iterrows():
      # if row['country_code']=="usa" or row['country_code']=="abw":  
      #   app.FTA(row['FTA_id'], row['has_fta'], row['signed_year'],row['inforced_year'],row['country_code'],"usa")
      app.FTA(row['FTA_id'], row['has_fta'], row['signed_year'],row['inforced_year'],row['country_code'],"usa")


    for idx, row in df_products.iterrows():
      app.product(row['section_id'],row['section_name'],row['category_id'],row['category_name'],row['item_id'],row['item_name'])
  

In [ ]:
df_trades=df_trades[~df_trades['exports_year'].isin([2000,2001,2002,2003,2004])]
df_trades

In [ ]:

tot_rows=0

for idx, row in df_trades.iterrows():
    tot_rows+=1
    print(tot_rows,row['exports_from'], row['exports_to'])
    app.trade(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],None,row['trade_value'],row['section_name'],row['section_id'])
    app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],None,row['trade_value'],row['section_name'],row['section_id'])

app.close()

In [ ]:
# for idx, row in df_trades.iterrows():
#     app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],None,row['trade_value'],row['section_name'],row['section_id'])

In [ ]:
# print('starting exports from')
# tot_rows=0
# for country in list(df_trades['exports_from'].unique())[:]:
#     print('working on this country from -->',country)
#     if country!="usa":
#         a=df_trades[df_trades['exports_from']==country ]
#         tot_rows+=len(a)
#         for idx, row in a.iterrows():
#             app.trade(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],None,row['trade_value'],row['section_name'],row['section_id'])
# print(tot_rows)



In [ ]:
# tot_rows=0

# # print('starting exports to')
# # for country in list(df_trades['exports_to'].unique())[:]:
# #     print('working on this country to-->',country)
# #     if country!="usa":
# #         a=df_trades[df_trades['exports_to']==country]
# #         tot_rows+=len(a)
# #         for idx, row in a.iterrows():
# #             app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],None,row['trade_value'],row['section_name'],row['section_id'])

# # print(tot_rows)
# # print('done')
# app.close()